In [ ]:
import re
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import time
import sqlite3
from urllib.parse import quote_plus
import yaml
from datetime import datetime
import os
from KBDownloader import search_swedish_newspapers, fetch_newspaper_data, save_checkpoint, load_checkpoint
from dotenv import load_dotenv

# Load the YAML configuration file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Load environment variables from .env file
load_dotenv()
kb_key = os.getenv('KB_API_KEY')

# Assign variables from the YAML configuration
venue_list = config['venue_list']
start_year = config['start_year']
years_to_crawl = config['years_to_crawl']
newspaper = config['newspaper']
db_path = config['db_path']
rate_limit = config['rate_limit']
num_composed_blocks = config.get('composed_blocks_context', 1)  # Default to 1 if not specified
years = config.get('years_to_crawl', [])  # Use 'years_to_crawl' instead of 'years'
if not years:
    raise ValueError("No years specified in the configuration file.")

# Define the newspaper collection IDs
NEWSPAPER_COLLECTION_IDS = {
    'Dagens nyheter': 'https://libris.kb.se/m5z2w4lz3m2zxpk#it',
    'Svenska Dagbladet': 'https://libris.kb.se/2ldhmx8d4mcrlq9#it',
    'Aftonbladet': 'https://libris.kb.se/dwpgqn5q03ft91j#it',
    'Dagligt Allehanda': 'https://libris.kb.se/9tmqzv3m32xfzcz#it',
    'Nya Dagligt Allehanda': 'https://libris.kb.se/2ldqsh7d0gp04wb#it'
}

# Get the correct collection ID for the specified newspaper
collection_id = NEWSPAPER_COLLECTION_IDS.get(newspaper)

if not collection_id:
    raise ValueError(f"Invalid newspaper name: {newspaper}")

# Ensure the database file exists
if not os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    conn.close()

# Create a connection to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create the newspaper_data table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS newspaper_data (
        Date TEXT,
        [Package ID] TEXT,
        Part INTEGER,
        Page INTEGER,
        [ComposedBlock ID] TEXT,
        [ComposedBlock Content] TEXT,
        [Raw API Result] TEXT,
        [Full Prompt] TEXT
    )
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

# Print out all the settings from the YAML configuration file
print("Configuration Settings:")
for key, value in config.items():
    print(f"{key}: {value}")
print(f"Collection ID: {collection_id}")

# Load the venue list
df = pd.read_excel(venue_list)

In [ ]:
# Load checkpoint if it exists
checkpoint = load_checkpoint()

# Main loop
for year in years:
    for half in range(2):
        if half == 0:
            from_date = datetime(year, 1, 1)
            to_date = datetime(year, 6, 30)
        else:
            from_date = datetime(year, 7, 1)
            to_date = datetime(year, 12, 31)

        print(f"Processing data from {from_date} to {to_date}")

        for index in range(len(df)):
            row = df.iloc[index]
            query = row['Lokal']

            try:
                result = fetch_newspaper_data(
                    query=query,
                    from_date=from_date.strftime('%Y-%m-%d'),
                    to_date=to_date.strftime('%Y-%m-%d'),
                    newspaper=collection_id,
                    config=config,
                    db_path=db_path,
                    kb_key=kb_key,
                    rate_limit=rate_limit,
                    num_composed_blocks=num_composed_blocks
                )

                if result.get('success'):
                    print(f"Processed query '{query}' successfully.")
                else:
                    print(f"Failed to process query '{query}': {result.get('message')}")

                # Save checkpoint after each query, successful or not
                save_checkpoint(year, half, index + 1)

            except Exception as e:
                print(f"Error processing query '{query}': {str(e)}")
                save_checkpoint(year, half, index)

        print(f"Waiting. Currently at {from_date} to {to_date}")
        time.sleep(0)  # in seconds

# Load SQL Database into Dataframe

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# SQL query to select all columns except 'Raw API Result'
query = """
SELECT Date, [Package ID], Part, Page, [ComposedBlock Content], [Full Prompt]
FROM newspaper_data
"""

# Read the query results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
df.head(10)

# Optional: If you want to save this to a CSV file for further analysis:
# df.to_csv('newspaper_data_summary.csv', index=False)

# STEP 2: Generate Prompts
Note! This is NECESSARY for moving to the next notebook

In [ ]:
# Load system message from the prompt file
with open(config['prompt_filepath'], 'r') as file:
    system_message_content = file.read().strip()

# Load the JSON schema from the file
with open(config['JSON_schema_path'], 'r') as file:
    json_schema = json.load(file)

def generate_full_prompt(row):
    date = row['Date']
    system_message = {"role": "system", "content": system_message_content.replace('{Newspaper_Date}', date)}
    user_content = str(row['ComposedBlock Content'])
    user_message = {"role": "user", "content": user_content}
    package_id = row['Package ID']
    part = row['Part']
    page = row['Page']

    # Get the row index to use as a sequential index
    row_index = row.name + 1

    custom_id = f"{package_id}-{part}-{page}-{row_index}"

    # Load the JSON schema from the file
    with open(config['JSON_schema_path'], 'r') as file:
        json_schema = json.load(file)

    # Prepare the full prompt JSON
    full_prompt = {
        "custom_id": custom_id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": config['llm_model'],
            "messages": [system_message, user_message],
            "max_tokens": config['max_tokens'],
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "response_data",
                    "strict": True,
                    "schema": json_schema
                }
            }
        }
    }

    return json.dumps(full_prompt)

# Generate the Full Prompt column
df['Full Prompt'] = df.apply(generate_full_prompt, axis=1)

df.head()

# Commit prompts to database

In [ ]:
import sqlite3
from sqlalchemy import create_engine

# Establish a connection to the database
engine = create_engine(f'sqlite:///{db_path}', echo=False)

# Update the existing table with the DataFrame containing the new 'Full Prompt' column
df.to_sql('newspaper_data', engine, if_exists='replace', index=False)

# Commit the changes and close the connection
conn = sqlite3.connect(db_path)
conn.commit()
conn.close()

print("Updated data committed to the 'newspaper_data' table. Rows: {}".format(len(df)))